In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
df = pd.read_csv('resultados/modelo.csv.zip')
df.fillna(0, inplace=True)
# rmsp = (df.rm == 20)
subnormal = (df.tipo_do_setor == 1)
df.loc[:, 'subnormal'] = 0
df.loc[subnormal, 'subnormal'] = 1

/tmp/ipykernel_23934/3615071609.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('resultados/modelo.csv.zip')


In [16]:
# Processa cada um dos municípios da sua Região MEtropolitana
rms = df.rm.unique()

for rm in rms:
    rm_filter = (df.rm == rm)
    # Verifica se existem mais de 20 setores subnormais
    # Processa os modelos, LDA, RandoForest, SVC
    # Salva as predicoes em atributos

In [33]:
r_subnormal = df.loc[(subnormal), ['rm', 'subnormal']].groupby('rm').count().reset_index()
r_subnormal_list = r_subnormal[(r_subnormal.subnormal < 20) | (r_subnormal.rm == 0.)].rm.to_list()

In [39]:
r_ufs = df[df.rm.isin(r_subnormal_list)].loc[(subnormal), ['uf', 'subnormal']].groupby('uf').count().reset_index()

In [42]:
r_ufs[r_ufs.subnormal >= 20]

,uf,subnormal
0,11,49
1,12,33
2,13,94
4,15,127
5,16,30
7,26,29
9,29,39
10,31,94
11,33,571
12,41,53


In [45]:
# Processa cada uma das UFs
df_uf = df[df.rm.isin(r_subnormal_list)]
ufs = r_ufs[r_ufs.subnormal >= 20].uf.to_list()

for uf in ufs:
    uf_filter = (df.uf == uf)
    print(uf)
    # Verifica se existem mais de 20 setores subnormais
    # Processa os modelos, LDA, RandoForest, SVC
    # Salva as predicoes em atributos

11
12
13
15
16
26
29
31
33
41
43
42
ES
SP
